Installing Haystack


In [ ]:
%%bash

pip install --upgrade pip
pip install farm-haystack[colab,inference]

In [45]:
from haystack.telemetry import tutorial_running

tutorial_running(1)

In [46]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [47]:
from haystack.document_stores import InMemoryDocumentStore

document_store = InMemoryDocumentStore(use_bm25=True)

INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0


In [48]:
doc_dir = "data"

In [51]:
import os
from haystack.pipelines.standard_pipelines import TextIndexingPipeline

files_to_index = [doc_dir + "/" + f for f in os.listdir(doc_dir)]
indexing_pipeline = TextIndexingPipeline(document_store)
indexing_pipeline.run_batch(file_paths=files_to_index)


INFO:haystack.pipelines.base:It seems that an indexing Pipeline is run, so using the nodes' run method instead of run_batch.
Updating BM25 representation...: 100%|██████████| 9/9 [00:00<00:00, 4387.35 docs/s]


{'documents': [<Document: {'content': "Cardiovascular health is a crucial aspect of overall well-being, as it directly impacts the\nproper functioning of the heart, the body's vital organ responsible for pumping blood and\nensuring the supply of oxygen and nutrients to all tissues. Despite significant\nadvancements in medical science, cardiovascular diseases continue to be a leading cause of\nglobal mortality, emphasizing the need for a comprehensive understanding of heart health.\nThis big document aims to explore various facets of cardiovascular health, including its\nanatomy, common diseases, risk factors, prevention strategies, diagnostic methods,\ntreatment options, the role of technology, psychological aspects, and initiatives for global\ncardiovascular health improvement.\nThe heart, a muscular organ situated in the chest, has a unique anatomy and physiology. It\nconsists of four chambers – two atria and two ventricles – that contract and relax in a\ncoordinated manner, allowing

In [ ]:
#from haystack.nodes import TfidfRetriever


In [52]:
from haystack.nodes import BM25Retriever

retriever = BM25Retriever(document_store=document_store)
# retriever = TfidfRetriever(document_store=document_store)


In [ ]:
from haystack.nodes import FARMReader

reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

In [54]:
from haystack.pipelines import ExtractiveQAPipeline

pipe = ExtractiveQAPipeline(reader, retriever)

In [ ]:
pip install gradio

In [56]:
import gradio as gr

In [ ]:
# prediction = pipe.run(
#     query="What are some example of cardiovascular exercises?", params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
# )

In [ ]:
# from pprint import pprint

# pprint(prediction)

In [ ]:
# from haystack.utils import print_answers

# print_answers(prediction, details="all")  ## Choose from `minimum`, `medium`, and `all`

In [60]:
def get_top_answer(prediction):
    answers = prediction['answers']
    if answers:
        highest_score = max(answer.score for answer in answers)
        highest_score_answer = next((answer for answer in answers if answer.score == highest_score), None)
        if highest_score_answer:
            return highest_score_answer.answer
    return "No answer found."

def answer_question(query):
    prediction = pipe.run(query=query, params={"Retriever": {"top_k": 3}, "Reader": {"top_k": 3}})
    return get_top_answer(prediction)


In [ ]:
# highest_score_answers = get_highest_score_answer(prediction)
# for answer in highest_score_answers:
#     print(answer.answer)


In [ ]:
# def answer_question(query):
#     prediction = pipe.run(query=query, params={"Retriever": {"top_k": 1}, "Reader": {"top_k": 1}})
#     highest_score_answers = get_highest_score_answer(prediction)
#     return [answer.answer for answer in highest_score_answers]


In [61]:
iface = gr.Interface(
    fn=answer_question,
    inputs="text",
    outputs="text",
    title="Question Answering Chatbot using Hugging face Model",
    description="Enter a question and get the answer from the pre-trained model.",
)

In [62]:
iface.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>